## Extract tar file

In [1]:
import tarfile

In [100]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
def extract(tar_url, extract_path='.'):
    tar = tarfile.open(tar_url, 'r')
    for item in tar:
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])
try:

    extract("semeval2023task3bundle-v3.tgz")
    print('Done')
except:
    name = os.path.basename(sys.argv[0])
    print(name[:name.rfind('.')], '<filename>')

Done


## Imports

In [4]:
!pip install pandas
!pip install sklearn
!pip install scikit-learn
!pip install datasets
!pip install datasets transformers[sentencepiece]
!pip install ipywidgets
!sudo apt-get install git-lfs
!pip install nltk
!pip install torchsampler
!pip install wandb

import torchsampler
import wandb
import pandas as pd
from tqdm import tqdm
import os
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.metrics import classification_report as report
from sklearn.feature_extraction.text import CountVectorizer
import argparse
import datasets
from datasets import Dataset
import numpy as np
from datasets import load_from_disk, load_metric
from transformers import AutoModelForSequenceClassification
from transformers import  Trainer, TrainingArguments
from transformers.trainer_utils import get_last_checkpoint
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from huggingface_hub import notebook_login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 150.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2936 sha256=e2963f20a85979ebc6a1d05dd3c343668b624f9cfc50e9f948b12f478699f94e
  Stored in directory: /root/.cache/pip/wheels/db/9f/0b/772886b624f84c138a5febb6966c89d374ab58c62bd65d109e
Successfully built sklearn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 146.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 73.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 127.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 112.3 MB/s eta 0:

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
notebook_login()

Token is valid.
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.huggingface/token
Login successful


In [8]:
from huggingface_hub import HfFolder
HF_TOKEN = HfFolder.get_token() 

## Load data 

In [9]:
en_folder_train = "data/en/train-articles-subtask-1/"
en_folder_dev = "data/en/dev-articles-subtask-1/"
en_labels_train_fn = "data/en/train-labels-subtask-1.txt"
en_labels_dev_fn = "data/en/dev-labels-subtask-1.txt"

it_folder_train = "data/it/train-articles-subtask-1/"
it_folder_dev = "data/it/dev-articles-subtask-1/"
it_labels_train_fn = "data/it/train-labels-subtask-1.txt"
it_labels_dev_fn = "data/it/dev-labels-subtask-1.txt"

ru_folder_train = "data/ru/train-articles-subtask-1/"
ru_folder_dev = "data/ru/dev-articles-subtask-1/"
ru_labels_train_fn = "data/ru/train-labels-subtask-1.txt"
ru_labels_dev_fn = "data/ru/dev-labels-subtask-1.txt"

fr_folder_train = "data/fr/train-articles-subtask-1/"
fr_folder_dev = "data/fr/dev-articles-subtask-1/"
fr_labels_train_fn = "data/fr/train-labels-subtask-1.txt"
fr_labels_dev_fn = "data/fr/dev-labels-subtask-1.txt"

ge_folder_train = "data/ge/train-articles-subtask-1/"
ge_folder_dev = "data/ge/dev-articles-subtask-1/"
ge_labels_train_fn = "data/ge/train-labels-subtask-1.txt"
ge_labels_dev_fn = "data/ge/dev-labels-subtask-1.txt"

po_folder_train = "data/po/train-articles-subtask-1/"
po_folder_dev = "data/po/dev-articles-subtask-1/"
po_labels_train_fn = "data/po/train-labels-subtask-1.txt"
po_labels_dev_fn = "data/po/dev-labels-subtask-1.txt"

In [10]:
def make_dataframe(input_folder, labels_folder=None):
    #MAKE TXT DATAFRAME
    text = []
    
    for fil in tqdm(filter(lambda x: x.endswith('.txt'), os.listdir(input_folder))):

        iD, txt = fil[7:].split('.')[0], open(input_folder +fil, 'r', encoding='utf-8').read() 
        text.append((iD, txt))

    df_text = pd.DataFrame(text, columns=['id','text']).set_index('id')

    df = df_text

    #MAKE LABEL DATAFRAME
    if labels_folder:
        labels = pd.read_csv(labels_folder, sep='\t', header=None)
        labels = labels.rename(columns={0:'id',1:'type'})
        labels.id = labels.id.apply(str)
        labels = labels.set_index('id')

        #JOIN
        df = labels.join(df_text)[['text','type']]

    return df

In [12]:
# #Read Data
print('Loading training...')
en_train = make_dataframe(en_folder_train, en_labels_train_fn)
it_train = make_dataframe(it_folder_train, it_labels_train_fn)
ru_train = make_dataframe(ru_folder_train, ru_labels_train_fn)
fr_train = make_dataframe(fr_folder_train, fr_labels_train_fn)
ge_train = make_dataframe(ge_folder_train, ge_labels_train_fn)
po_train = make_dataframe(po_folder_train, po_labels_train_fn)

print('Loading dev...')
en_test = make_dataframe(en_folder_dev, en_labels_dev_fn)
it_test = make_dataframe(it_folder_dev, it_labels_dev_fn)
ru_test = make_dataframe(ru_folder_dev, ru_labels_dev_fn)
fr_test = make_dataframe(fr_folder_dev, fr_labels_dev_fn)
ge_test = make_dataframe(ge_folder_dev, ge_labels_dev_fn)
po_test = make_dataframe(po_folder_dev, po_labels_dev_fn)

# train = pd.read_csv("babe.csv", sep=';')

en_train.head()

Loading training...


433it [00:00, 34366.53it/s]
226it [00:00, 40381.39it/s]
142it [00:00, 26206.33it/s]
157it [00:00, 32892.39it/s]
132it [00:00, 35563.21it/s]
144it [00:00, 30965.38it/s]


Loading dev...


83it [00:00, 38573.65it/s]
77it [00:00, 38925.08it/s]
49it [00:00, 26477.83it/s]
54it [00:00, 32167.65it/s]
45it [00:00, 32385.67it/s]
50it [00:00, 28111.96it/s]


,text,type
id,,
833042063,Chelsea Handler Admits She’s ‘Very Sexually At...,satire
832959523,How Theresa May Botched\n\nThose were the time...,satire
833039623,Robert Mueller III Rests His Case—Dems NEVER W...,satire
833032367,Robert Mueller Not Recommending Any More Indic...,satire
814777937,The Far Right Is Trying to Co-opt the Yellow V...,satire


In [13]:
train = pd.concat([en_train, it_train, ru_train, fr_train, ge_train, po_train])
test = pd.concat([en_test, it_test, ru_test, fr_test, ge_test, po_test])

## Preprocess text

In [14]:
train = train.dropna()
test = test.dropna()
train.head()

,text,type
id,,
833042063,Chelsea Handler Admits She’s ‘Very Sexually At...,satire
832959523,How Theresa May Botched\n\nThose were the time...,satire
833039623,Robert Mueller III Rests His Case—Dems NEVER W...,satire
833032367,Robert Mueller Not Recommending Any More Indic...,satire
814777937,The Far Right Is Trying to Co-opt the Yellow V...,satire


In [67]:
def process(paragraph):
    ps = PorterStemmer()
    corpus = []
    
    paragraph = str(paragraph).replace("-", " ")
    cleaned = re.sub('[^a-zA-Z]', ' ', paragraph)
    cleaned = cleaned.lower().split()
    
    cleaned = [ps.stem(word) for word in cleaned if not word in stopwords.words('english')]
    cleaned = ' '.join(cleaned)
    
    return cleaned

In [15]:
# train["text"] = train["text"].apply(lambda x: process(x))
# test["text"] = test["text"].apply(lambda x: process(x))

In [16]:
train.rename(columns={'label_opinion': 'type'}, inplace=True)
test.rename(columns={'label_opinion': 'type'}, inplace=True)

train.head(10)

,text,type
id,,
833042063,Chelsea Handler Admits She’s ‘Very Sexually At...,satire
832959523,How Theresa May Botched\n\nThose were the time...,satire
833039623,Robert Mueller III Rests His Case—Dems NEVER W...,satire
833032367,Robert Mueller Not Recommending Any More Indic...,satire
814777937,The Far Right Is Trying to Co-opt the Yellow V...,satire
821744708,‘Special place in hell’ for those who promoted...,satire
833036489,Bill Maher says he doesn't need Mueller report...,satire
707566605,Madagascar Outbreak: It Is ‘Inevitable’ The Pl...,opinion
708561738,How Do You Like Paying For Sexual Harassment S...,satire


In [17]:
train.shape

(1234, 2)

In [18]:
from datasets import Value, ClassLabel, Features

data = pd.concat([train, test])

# train_dataset = Dataset.from_pandas(train)
# train_dataset = train_dataset.class_encode_column("type")

# test_dataset = Dataset.from_pandas(test)
# test_dataset = test_dataset.class_encode_column("type")


dataset = Dataset.from_pandas(data)
dataset = dataset.class_encode_column("type")

train_test = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42, stratify_by_column="type")

dataset = datasets.DatasetDict(
    {
        "train": train_test['train'],
        "test": train_test['test'],
    }
)

Casting to class labels:   0%|          | 0/2 [00:00<?, ?ba/s]

In [20]:
train_dataset = dataset['train']
test_dataset = dataset['test']

## Preprocessing

In [21]:
from transformers import AutoTokenizer

checkpoint = 'xlm-roberta-base'

In [22]:
max_input_length = 512

# shuffle dataset
train_dataset = train_dataset.shuffle()
test_dataset = test_dataset.shuffle()

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
#                                           cache_dir=CACHE_PRETRAINED
                                         )

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=max_input_length)

# collator
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset = train_dataset.remove_columns(['id'])
train_dataset =  train_dataset.rename_column("type", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

test_dataset = test_dataset.remove_columns(['id'])
test_dataset =  test_dataset.rename_column("type", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(train_dataset)
print(test_dataset)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1273
})
Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 319
})


In [23]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
#                                           cache_dir=CACHE_PRETRAINED
                                         )

# tokenizer helper function
def tokenize(batch):
#     return tokenizer(batch['review_body'], truncation=True, max_length=max_input_length)
    return tokenizer(batch['text'], truncation=True, max_length=max_input_length)

# collator
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
tokenizer("SemEval23 is all you need")

{'input_ids': [0, 10232, 647, 1405, 3742, 83, 756, 398, 3871, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [25]:
training_input_path = f'dataset/train'
train_dataset.save_to_disk(training_input_path)

test_input_path = f'dataset/test'
test_dataset.save_to_disk(test_input_path)

Saving the dataset (0/1 shards):   0%|          | 0/1273 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/319 [00:00<?, ? examples/s]

## Training

In [26]:
hub_model_id = f'SEMEVAL23_TASK3_SUBTASK1_MULTI'

In [27]:
 # load datasets
train_dataset = load_from_disk(training_input_path)
test_dataset = load_from_disk(test_input_path)

print(f"loaded train_dataset length is: {len(train_dataset)}")
print(f"loaded test_dataset length is: {len(test_dataset)}")

# define metrics and metrics function
metric = load_metric("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

# Prepare model labels - useful in inference API
labels = train_dataset.features["labels"].names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

loaded train_dataset length is: 1273
loaded test_dataset length is: 319


In [28]:
labels

['opinion', 'reporting', 'satire']

In [29]:
# download model from model hub
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    num_labels=num_labels, 
    ignore_mismatched_sizes=True,
    label2id=label2id, 
    id2label=id2label,
#     cache_dir=CACHE_PRETRAINED
)
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [36]:
# define training args
training_args = TrainingArguments(
        hub_model_id,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        num_train_epochs=10,
        logging_steps = 1,  
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        warmup_steps=0,
        eval_steps=2,
        fp16=True,
        learning_rate=2e-5,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        # push to hub parameters
        push_to_hub=True,
        hub_strategy='every_save',
        hub_model_id=hub_model_id,
        hub_token=HF_TOKEN,
#         report_to="wandb",
    )

In [31]:
train_dataset

Dataset({
    features: ['text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 1273
})

In [32]:
from torch.utils.data import DataLoader
from torchsampler import ImbalancedDatasetSampler

class ImbalancedSamplerTrainer(Trainer):
    def get_train_dataloader(self) -> DataLoader:
        train_dataset = self.train_dataset

        def get_label(dataset):
            return dataset["labels"]

        train_sampler = ImbalancedDatasetSampler(
            train_dataset, callback_get_label=get_label
        )

        return DataLoader(
            train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=train_sampler,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

In [33]:
# try:
#     wandb.login(key="f83b2236e0a095919b77eb0cde145946fdabc4a7")
#     anony=None
# except:
#     anony = "must"
#     print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')
    
    
    
# wandb.init(project="SEMEVAL23_TASK3",name=f"SUBTASK1_EN")

In [34]:
%env WANDB_LOG_MODEL=true

%env WANDB_PROJECT=SEMEVAL23_TASK3
%env TASK_NAME=SUBTASK1_EN

env: WANDB_LOG_MODEL=true
env: WANDB_PROJECT=SEMEVAL23_TASK3
env: TASK_NAME=SUBTASK1_EN


In [37]:
# create Trainer instance
trainer = ImbalancedSamplerTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# train model
trainer.train()

# evaluate model
eval_result = trainer.evaluate(eval_dataset=test_dataset)

Cloning https://huggingface.co/franfj/SEMEVAL23_TASK3_SUBTASK1_MULTI into local empty directory.
Using cuda_amp half precision backend
***** Running training *****
  Num examples = 1273
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 400
  Number of trainable parameters = 278045955
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,0.986900,0.951657,0.424414
2,0.556900,0.736543,0.551421
3,0.455900,0.782863,0.534836
4,0.201800,0.628121,0.624091
5,0.112500,0.929075,0.561389
6,0.259400,1.036723,0.588355
7,0.188300,0.768297,0.631737
8,0.108300,0.865980,0.610554
9,0.065900,0.869534,0.618896
10,0.255400,1.009395,0.583870


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 319
  Batch size = 32
Saving model checkpoint to SEMEVAL23_TASK3_SUBTASK1_MULTI/checkpoint-40
Configuration saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/checkpoint-40/config.json
Model weights saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/checkpoint-40/pytorch_model.bin
tokenizer config file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/checkpoint-40/tokenizer_config.json
Special tokens file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/checkpoint-40/special_tokens_map.json
tokenizer config file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/tokenizer_config.json
Special tokens file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This 

Upload file pytorch_model.bin:   0%|          | 3.30k/1.04G [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/franfj/SEMEVAL23_TASK3_SUBTASK1_MULTI
   bc8508a..7a3c337  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}}
To https://huggingface.co/franfj/SEMEVAL23_TASK3_SUBTASK1_MULTI
   7a3c337..76eb3c4  main -> main

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 319
  Batch size = 32


In [38]:
# save best model, metrics and create model card
trainer.create_model_card(model_name=hub_model_id)
trainer.push_to_hub()

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.6317366037874205}]}
Saving model checkpoint to SEMEVAL23_TASK3_SUBTASK1_MULTI
Configuration saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/config.json
Model weights saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/pytorch_model.bin
tokenizer config file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/tokenizer_config.json
Special tokens file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/special_tokens_map.json
To https://huggingface.co/franfj/SEMEVAL23_TASK3_SUBTASK1_MULTI
   76eb3c4..56edd0e  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.6317366037874205}]}


'https://huggingface.co/franfj/SEMEVAL23_TASK3_SUBTASK1_MULTI/commit/56edd0e60d58a665aa38923cdca2eca346dbe6ab'

In [39]:
# Saves the model
trainer.save_model('model/')

Saving model checkpoint to model/
Configuration saved in model/config.json
Model weights saved in model/pytorch_model.bin
tokenizer config file saved in model/tokenizer_config.json
Special tokens file saved in model/special_tokens_map.json
Saving model checkpoint to SEMEVAL23_TASK3_SUBTASK1_MULTI
Configuration saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/config.json
Model weights saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/pytorch_model.bin
tokenizer config file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/tokenizer_config.json
Special tokens file saved in SEMEVAL23_TASK3_SUBTASK1_MULTI/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.6317366037874205}]}


In [61]:
def getTestInstances(folder_dev):
    test = make_dataframe(folder_dev)
    texts = test['text'].tolist()

    for i in range(0, len(texts)):
        texts[i] = texts[i][:512]
        
    return texts

In [44]:
from transformers import TextClassificationPipeline

MODEL_NAME = hub_model_id
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file SEMEVAL23_TASK3_SUBTASK1_MULTI/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "SEMEVAL23_TASK3_SUBTASK1_MULTI",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "opinion",
    "1": "reporting",
    "2": "satire"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "opinion": "0",
    "reporting": "1",
    "satire": "2"
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "positio

In [101]:
texts = getTestInstances(po_folder_dev)

50it [00:00, 19919.76it/s]


In [102]:
prediction = pipe(texts, return_all_scores=False)

In [103]:
t = make_dataframe(po_folder_dev)

50it [00:00, 614.49it/s]


In [104]:
t['prediction'] = prediction

In [105]:
t['prediction'] = t['prediction'].apply(lambda x: x['label'])
t.head()

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/opt/conda/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


,text,prediction
id,,
2564,Grzegorz Braun i naukowcy ws. pokoju na Ukrain...,opinion
25100,Czy Polacy są gotowi na czterodniowy tydzień p...,satire
2566,Ryszard Czarnecki: Niemieckiej „Ostpolitik” ży...,opinion
2598,Kolonializm ciągle trwa. I jest jedną z przycz...,opinion
25129,Napięcie na granicy Kosowa z Serbią. Protest e...,reporting


In [106]:
out = t
del out['text']

In [107]:
out.to_csv('SEMEVAL23_TASK3_SUBTASK1_MULTI_PO.txt', sep="\t", header=False)

In [108]:
out.value_counts()

prediction
satire        19
reporting     16
opinion       15
dtype: int64